# Google Trends in AQL
In this notebook, we inspect the most prominent queries with respect to certain time windows and compare them to the most frequent queries from google in that same time window.
First, we look at the most frequent queries in a year. We examine the years from 1999 up to 2022.

In [2]:
import pandas as pd

annual_google_trends = pd.read_csv("/mnt/ceph/storage/data-in-progress/data-teaching/theses/thesis-schneg/google_trends/google_trends_total.csv")

annual_google_trends.head()

,query,score,year
0,google,100,2022
1,youtube,77,2022
2,you,69,2022
3,video,67,2022
4,facebook,62,2022


In [5]:
annual_aql_trends = pd.read_parquet("/mnt/ceph/storage/data-in-progress/data-teaching/theses/thesis-schneg/analysis_data/analysis/aql-get-annual-top-queries-special")

annual_aql_trends[annual_aql_trends['year'] == 2020]

,year,serp_query_text_url,count()
525,2020,openeuler,44268
526,2020,from:EPAWater AND #greenwater,18117
527,2020,#zesummendoheem/,15891
528,2020,#nolaopen,14523
529,2020,Avril Lavigne Knockin’ on Heaven’s Door,7272
530,2020,Cat Stevens Peace Train,7214
531,2020,John Lennon Imagine,7175
532,2020,John Mellencamp Peaceful World,6915
533,2020,The Rennas Peace In My Heart,6840
534,2020,Jann Arden Fighting for the World,6547
